In [1]:
import pandas as pd
import numpy as np
import json

## Explore Demographics of U.S. Cities Data

I obtained demogrpahic data on cities from [Open Data Soft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/information/), which obtained their dataset from the U.S. Census Bureau for 2015. "This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000." The U.S. Census Bureau data is open and available to the public.

Note: The JSON file, when read, has three columns: `datasetid`, `recordid`, and `fields`. `datasetid` is `us-cities-demographics` and `recordid` is a unique identifier for each city. `fields` is a dictionary that contains all of the demographic data we care about. So, when I create the dataframe initially, I explode the fields column, which gives me all of columns I actually care about to explore.

In [2]:
city_df = pd.read_json('data/demographic_data/us-cities-demographics.json')
city_df = city_df['fields'].apply(pd.Series)

In [3]:
city_df.head()

,total_population,female_population,count,foreign_born,state_code,average_household_size,city,race,male_population,median_age,number_of_veterans,state
0,389955,197601.0,8791,40270.0,KS,2.56,Wichita,American Indian and Alaska Native,192354.0,34.6,23978.0,Kansas
1,120207,59581.0,22304,19652.0,PA,2.67,Allen,Black or African-American,60626.0,33.5,5691.0,Pennsylvania
2,84662,41227.0,8454,25675.0,CT,2.74,Danbury,Black or African-American,43435.0,37.3,3752.0,Connecticut
3,654596,340365.0,67526,88193.0,TN,2.39,Nashville,Hispanic or Latino,314231.0,34.1,27942.0,Tennessee
4,128877,63936.0,11013,44003.0,CT,2.70,Stamford,Asian,64941.0,35.4,2269.0,Connecticut


In [4]:
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   total_population        2891 non-null   int64  
 1   female_population       2888 non-null   float64
 2   count                   2891 non-null   int64  
 3   foreign_born            2878 non-null   float64
 4   state_code              2891 non-null   object 
 5   average_household_size  2875 non-null   float64
 6   city                    2891 non-null   object 
 7   race                    2891 non-null   object 
 8   male_population         2888 non-null   float64
 9   median_age              2891 non-null   float64
 10  number_of_veterans      2878 non-null   float64
 11  state                   2891 non-null   object 
dtypes: float64(6), int64(2), object(4)
memory usage: 271.2+ KB


In [5]:
city_df['city_state'] = city_df['city'] + ', ' + city_df['state']

In [6]:
unique_city_count = city_df.city_state.nunique()
unique_city_count

596

In [7]:
state_summary = city_df.groupby('state')\
    .agg({'city': 'count', 'average_household_size': 'mean', 'median_age': 'mean', 'foreign_born': 'max', 'number_of_veterans': 'max'})\
    .round({'average_household_size': 2, 'median_age': 2})\
    .rename(columns={'city': 'city_count'})
state_summary

,city_count,average_household_size,median_age,foreign_born,number_of_veterans
state,,,,,
Alabama,34,2.43,36.16,12691.0,16637.0
Alaska,5,2.77,32.20,33258.0,27492.0
Arizona,80,2.77,35.04,300702.0,72388.0
Arkansas,29,2.53,32.74,19969.0,12343.0
California,676,3.10,36.17,1485425.0,92489.0
Colorado,80,2.56,35.82,113222.0,49291.0
Connecticut,39,2.67,35.00,44475.0,4646.0
Delaware,5,2.45,36.40,3336.0,3063.0
District of Columbia,5,2.24,33.80,95117.0,25963.0


In [8]:
us_stats_by_race = city_df.groupby('race')\
    .agg({'count': 'sum', 'average_household_size': 'mean', 'median_age': 'mean'})\
    .round({'average_household_size': 2, 'median_age': 2})
us_stats_by_race

,count,average_household_size,median_age
race,,,
American Indian and Alaska Native,1782553,2.73,35.38
Asian,11159489,2.74,35.51
Black or African-American,22479120,2.74,35.51
Hispanic or Latino,31864940,2.75,35.56
White,74268170,2.75,35.50


## Explore Yelp Data

The Yelp datasets are large. To get a good idea of how they're structured and what's in them, I take a subset of the first 500 rows to get acquainted with the data.

In [9]:
business_list = []
with open('data/yelp_data/yelp_academic_dataset_business.json', encoding='utf-8') as f:
    for i, json_obj in enumerate(f):
        business_dict = json.loads(json_obj)
        business_list.append(business_dict)
        if i == 500:
            break

business_df = pd.json_normalize(business_list)

In [10]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.GoodForDancing,attributes.BestNights,attributes.Music,attributes.BYOB,attributes.CoatCheck,attributes.Smoking,attributes.DriveThru,attributes.BYOBCorkage,attributes.Corkage,attributes.RestaurantsCounterService
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 57 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   business_id                            501 non-null    object 
 1   name                                   501 non-null    object 
 2   address                                501 non-null    object 
 3   city                                   501 non-null    object 
 4   state                                  501 non-null    object 
 5   postal_code                            501 non-null    object 
 6   latitude                               501 non-null    float64
 7   longitude                              501 non-null    float64
 8   stars                                  501 non-null    float64
 9   review_count                           501 non-null    int64  
 10  is_open                                501 non-null    int64  
 11  catego

In [12]:
stars_count = business_df.groupby('stars')['stars'].count()
stars_count

stars
1.0      7
1.5     17
2.0     22
2.5     42
3.0     59
3.5     93
4.0    105
4.5    106
5.0     50
Name: stars, dtype: int64

In [13]:
avg_review_counts_by_state = business_df.groupby('state')['review_count'].mean().round(2)
avg_review_counts_by_state

state
BC    23.34
CO    52.60
FL    56.69
GA    63.17
MA    59.21
OH    61.56
OR    51.77
TX    38.00
WA    16.50
Name: review_count, dtype: float64

In [14]:
review_df = pd.read_json('data/yelp_data/yelp_academic_dataset_review.json', lines=True, nrows=500)

In [15]:
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01


In [16]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   review_id    500 non-null    object        
 1   user_id      500 non-null    object        
 2   business_id  500 non-null    object        
 3   stars        500 non-null    int64         
 4   useful       500 non-null    int64         
 5   funny        500 non-null    int64         
 6   cool         500 non-null    int64         
 7   text         500 non-null    object        
 8   date         500 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 35.3+ KB


In [17]:
avg_stats_by_year = review_df.groupby(review_df.date.dt.year).agg({
    'stars': 'mean',
    'useful': 'mean',
    'funny': 'mean',
    'cool': 'mean'
}).round(2)
avg_stats_by_year

,stars,useful,funny,cool
date,,,,
2004,2.00,0.00,0.00,0.00
2005,4.00,0.50,0.00,0.00
2006,3.17,0.67,0.67,0.50
2007,3.67,1.67,0.17,1.00
2008,4.64,0.82,0.18,0.64
2009,3.36,1.91,0.82,0.55
2010,3.67,1.50,1.00,1.07
2011,3.77,1.23,0.09,0.27
2012,3.93,0.59,0.30,0.15


In [18]:
user_df = pd.read_json('data/yelp_data/yelp_academic_dataset_user.json', lines=True, nrows=500)

In [19]:
user_df.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,190,361,147,1212,5691,2541,2541,815,323
1,dIIKEfOgo0KqUfGQvGikPg,Gabi,2136,2007-08-10 19:01:51,21272,10289,18046,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","XPzYf9_mwG2eXYP2BAGSTA, 2LooM5dcIk2o01nftYdPIg...",1025,...,87,94,232,96,1187,3293,2205,2205,472,294
2,D6ErcUnFALnCQN4b1W_TlA,Jason,119,2007-02-07 15:47:53,188,128,130,"2010,2011","GfB6sC4NJQvSI2ewbQrDNA, jhZtzZNNZJOU2YSZ6jPlXQ...",16,...,1,3,0,0,5,20,31,31,3,1
3,JnPIjvC0cmooNDfsa9BmXg,Kat,987,2009-02-09 16:14:29,7234,4722,4035,"2009,2010,2011,2012,2013,2014","HQZPQhKMwRAyS6BCselVWQ, kP2U1s_sjQfHO9grxiyDTA...",420,...,129,93,219,90,1120,4510,1566,1566,391,326
4,37Hc8hr3cw0iHLoPzLK6Ow,Christine,495,2008-03-03 04:57:05,1577,727,1124,"2009,2010,2011","-Q88pZUcrfN0BLBDp-bkAQ, etPn4Pv1Gc4cRZjRgB_BOw...",47,...,19,32,16,15,77,131,310,310,98,44


In [20]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             500 non-null    object 
 1   name                500 non-null    object 
 2   review_count        500 non-null    int64  
 3   yelping_since       500 non-null    object 
 4   useful              500 non-null    int64  
 5   funny               500 non-null    int64  
 6   cool                500 non-null    int64  
 7   elite               500 non-null    object 
 8   friends             500 non-null    object 
 9   fans                500 non-null    int64  
 10  average_stars       500 non-null    float64
 11  compliment_hot      500 non-null    int64  
 12  compliment_more     500 non-null    int64  
 13  compliment_profile  500 non-null    int64  
 14  compliment_cute     500 non-null    int64  
 15  compliment_list     500 non-null    int64  
 16  complime

In [21]:
top_reviewers = user_df.sort_values('review_count', ascending=False).head()
top_reviewers

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
270,P5bUL3Engv-2z6kKohB6qQ,Kim,9906,2006-05-31 21:27:42,27201,10572,21051,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","hfY2Soz8LWSA0oRVcYuRmg, Y6_gZa4jVXv0d4qiMbbycw...",809,...,128,111,47,34,507,1070,1188,1188,474,246
265,syvwUKQJ4OYfmL_ixVLMeQ,Tina,4308,2008-06-30 12:39:30,18558,8538,11953,"2008,2009,2010,2011,2012,2013,2014,2015,2016,2...","Z6ctuZkS5H2fRp9sbd-Ekw, cAmd-fttNSq-8zgreIy2YQ...",555,...,75,65,39,9,340,578,1076,1076,429,185
59,CQUDh80m48xnzUkx-X5NAw,David,4205,2008-12-29 21:03:01,21059,8906,14640,"2009,2010,2011,2012,2013,2014","WnJlu4mpNtVxNQ2SM6GmvQ, 3BqKBuvY09lissdY_soI6w...",575,...,149,140,66,287,1034,1529,1572,1572,492,180
172,YttDgOC9AlM4HcAlDsbB2A,Phil,3531,2009-09-02 15:30:08,35695,24357,30221,"2011,2012,2013,2014,2015,2016,2017,2018,2019,2...","QQ82r5VjULzbWc7aIHajnQ, D4Eznn83MRWUK39n5WI0JA...",792,...,260,279,142,154,2387,7132,5113,5113,1869,1014
104,gjhzKWsqCIrpEd9pevbKZw,Miriam,3192,2005-09-21 16:54:53,19635,8160,15711,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","mbSAI_HwP_6zoXKKhHPdwA, YcPI5wlTN-yKMcfzVXylBg...",1190,...,186,195,393,160,1240,2208,2648,2648,638,284


In [22]:
most_useful = user_df.sort_values('useful', ascending=False).head()
most_useful

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
50,8ZryN_S-n48g6rsa3W3QtQ,Dani,2689,2008-10-27 21:47:03,56628,45720,52331,"2009,2010,2012,2013,2014,2015,2016,2017,2018,2...","E0x8ew2yWRHcVdEMPI7kkg, v0O7miDEH4XX2diEr7ij2w...",392,...,630,352,105,136,1807,2816,4928,4928,2520,1936
343,hizGc5W1tBHPghM5YKCAtg,Katie,1754,2009-05-22 20:19:55,46534,31112,37964,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","U_U0yg03tOYHQlnCQhZAZg, ffarmP6gowdXoTxQARKU7A...",3511,...,424,416,373,252,3104,17426,7769,7769,2524,1620
227,VHdY6oG2JPVNjihWhOooAQ,Jessica,2049,2005-07-21 01:16:04,44981,28023,42647,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","7_HzbEaesggsuTGiDGJLpQ, 3zBJUlWtPNoZ0uN83ODbyg...",2463,...,393,388,963,152,2007,13653,7074,7074,2257,539
329,QnIrewrXxFnssw_gG1gF-Q,Sarah,2845,2010-07-10 18:54:05,43203,22264,36762,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...","NFf4OIoG15sM4qRz2olwgg, tqZiHXN64sZ721outuzK5Q...",1299,...,237,190,164,95,1290,3663,3121,3121,1074,1981
184,NOUfyJW-BAo_-Cbfo8edww,Lolia,2469,2007-05-13 03:31:10,41337,29326,36993,"2007,2008,2009","Qa2NoSc6DphQArihBgY9Gg, l-zgWG-brcQ4iRev-pfrsg...",605,...,2211,4123,2487,1390,4812,11124,24495,24495,7195,12801


In [23]:
funniest = user_df.sort_values('funny', ascending=False).head()
funniest

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
50,8ZryN_S-n48g6rsa3W3QtQ,Dani,2689,2008-10-27 21:47:03,56628,45720,52331,"2009,2010,2012,2013,2014,2015,2016,2017,2018,2...","E0x8ew2yWRHcVdEMPI7kkg, v0O7miDEH4XX2diEr7ij2w...",392,...,630,352,105,136,1807,2816,4928,4928,2520,1936
343,hizGc5W1tBHPghM5YKCAtg,Katie,1754,2009-05-22 20:19:55,46534,31112,37964,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","U_U0yg03tOYHQlnCQhZAZg, ffarmP6gowdXoTxQARKU7A...",3511,...,424,416,373,252,3104,17426,7769,7769,2524,1620
184,NOUfyJW-BAo_-Cbfo8edww,Lolia,2469,2007-05-13 03:31:10,41337,29326,36993,"2007,2008,2009","Qa2NoSc6DphQArihBgY9Gg, l-zgWG-brcQ4iRev-pfrsg...",605,...,2211,4123,2487,1390,4812,11124,24495,24495,7195,12801
227,VHdY6oG2JPVNjihWhOooAQ,Jessica,2049,2005-07-21 01:16:04,44981,28023,42647,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","7_HzbEaesggsuTGiDGJLpQ, 3zBJUlWtPNoZ0uN83ODbyg...",2463,...,393,388,963,152,2007,13653,7074,7074,2257,539
406,B7ecAeAIrXg7sgmabS38pg,Stephy,2602,2006-05-12 06:07:32,24986,25943,23273,"2006,2007,2008,2010","Vjk5FemZ5rwJuJ7Sznd3fA, Bk594MkOK2JrTIbvVVXI6w...",1371,...,292,223,600,142,1697,2360,3233,3233,973,348


## Tax Foundation Data

In [24]:
tax_climate_df = pd.read_csv('data/tax_climate/state_business_tax_climate.csv')

In [25]:
tax_climate_df

,State,Overall Rank,Corporate Tax Rank,Individual Income Tax Rank,Sales Tax Rank,Property Tax Rank,Unemployment Insurance Tax Rank
0,Alabama,41,23,30,50,19,14
1,Alaska,3,26,1,5,22,45
2,Arizona,24,22,17,40,11,8
3,Arkansas,45,34,41,46,25,23
4,California,49,28,49,45,14,21
5,Colorado,21,10,14,36,32,41
6,Connecticut,47,27,44,26,50,22
7,Delaware,13,50,42,2,4,3
8,Florida,4,6,1,21,13,2
9,Georgia,31,7,36,27,24,39


In [26]:
tax_climate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   State                            51 non-null     object
 1   Overall Rank                     51 non-null     int64 
 2   Corporate Tax Rank               51 non-null     int64 
 3   Individual Income Tax Rank       51 non-null     int64 
 4   Sales Tax Rank                   51 non-null     int64 
 5   Property Tax Rank                51 non-null     int64 
 6   Unemployment Insurance Tax Rank  51 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 2.9+ KB
